## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [17]:
##importing necessary tools
import pandas as pd
import seaborn as sns
import numpy as np
import os
from glob import glob
from datetime import datetime

In [32]:
csv_files = glob("./zippedData/*.csv.gz")
csv_files

csv_files_dict = {}
for filename in csv_files:
    filename_cleaned = os.path.basename(filename).replace(".csv", "").replace(".", "_")
    filename_df = pd.read_csv(filename, index_col=0)
    csv_files_dict[filename_cleaned] = filename_df
rt_info = pd.read_csv("zippedData/rt.movie_info.tsv.gz", delimiter='\t')
rt_reviews = pd.read_csv("zippedData/rt.reviews.tsv.gz", delimiter='\t',encoding='latin1')
imdb_title_basics = csv_files_dict['imdb_title_basics_gz']
tn_movie_budgets = csv_files_dict['tn_movie_budgets_gz']
imdb_title_ratings = csv_files_dict['imdb_title_ratings_gz']
rt_combined = rt_info.merge(rt_reviews, how='inner',on='id')
title_and_rating_df = imdb_title_ratings.join(imdb_title_basics, how = 'inner')
title_and_budget = tn_movie_budgets.merge(
imdb_title_basics, how='inner', left_on = 'movie', right_on = 'primary_title')

In [33]:


rt_combined.drop(columns = ['synopsis', 'theater_date','dvd_date', 'studio',
'runtime','review'], inplace=True)

rt_top_critics = rt_combined[rt_combined['top_critic'] !=0].dropna()


In [34]:
directors_sorted = rt_combined['director'].value_counts(ascending=False).to_frame()
top_60_directors = directors_sorted[:60]
top_60_directors = top_60_directors.reset_index().rename(
columns={'index':'director','director':'rating_count'})

In [35]:
top_directors_reviews = top_60_directors.merge(rt_top_critics)
director_ratings = top_directors_reviews.groupby(['director','fresh']).count()
director_ratings = director_ratings['rating_count'].unstack('fresh')
director_ratings['fresh/rotten'] = director_ratings[
'fresh']/director_ratings['rotten']

top_15_directors = director_ratings.sort_values(
'fresh/rotten', ascending=False)[0:15]


In [36]:
title_and_budget.drop(columns=[
'start_year', 
'movie', 
'original_title'], inplace=True)

In [37]:
title_and_budget['production_budget'] = title_and_budget[
'production_budget'].str.replace("$","").str.replace(",","").astype(int)

title_and_budget['domestic_gross'] = title_and_budget[
'domestic_gross'].str.replace("$","").str.replace(",","").astype(int)

title_and_budget['worldwide_gross'] = title_and_budget[
'worldwide_gross'].str.replace("$","").str.replace(",","").astype(int)


In [38]:
title_and_budget.describe()
title_and_budget = title_and_budget[
title_and_budget['production_budget'] >= 16000000]

In [39]:
title_and_budget['release_date'] = pd.to_datetime(
title_and_budget['release_date'])
title_and_budget['release_month'] = title_and_budget['release_date'].dt.month

In [40]:
title_and_budget['profit'] = title_and_budget[
'worldwide_gross'] - title_and_budget['production_budget']

title_and_budget['ROI'] = title_and_budget[
'profit']/title_and_budget['production_budget']

    


In [41]:
title_and_budget = title_and_budget[title_and_budget['production_budget'] !=0]
title_and_budget = title_and_budget[title_and_budget['domestic_gross'] !=0]
title_and_budget = title_and_budget[title_and_budget['worldwide_gross'] !=0]

In [42]:
title_and_budget['genres'] = title_and_budget['genres'].str.split(",")

In [43]:
title_budget_exploded = title_and_budget.explode('genres')


In [44]:
genres_grouped = title_budget_exploded.groupby(
'genres').mean().sort_values('ROI', ascending=False)

months_grouped = title_and_budget.groupby(
'release_month').mean().sort_values('worldwide_gross', ascending=False)